In [ ]:
pip install clickhouse-driver

In [ ]:
pip install psycopg

In [ ]:
pip install "psycopg[binary,pool]"

In [1]:
from connector import spark_connector

In [2]:
import pandas as pd
import numpy as np
from pyspark.sql import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [3]:
spark_connector?

Init signature:
spark_connector(
    exec_inst: int = 100,
    exec_cores: int = 5,
    aloc_max: int = 500,
    memory_exec: int = 25,
    name: object = 'utopialvo',
    rootdir: object = 'remote_dir_in_hdft',
    enable_graphframes: bool = False,
    enable_clickhouse: bool = False,
    enable_postgres: bool = False,
    dynamic_allocation: bool = False,
    intensive_mode: bool = False,
    marshal_mode: bool = True,
) -> None
Docstring:      Класс стандартного подключения к Spark
Init docstring:         
File:           ~/work/Spark_conn/connector.py
Type:           type
Subclasses:     

In [4]:
con = spark_connector(exec_inst=10, exec_cores=5, memory_exec=15, enable_graphframes=True, enable_clickhouse=True, enable_postgres=True, dynamic_allocation=False)

In [ ]:
con.spark

In [6]:
from graphframes import GraphFrame

In [7]:
v = con.spark.createDataFrame([
    (1, 'Anton', 23),
    (2, 'Anna', 27),
    (3, 'Andry', 24),
    (4, 'Alex', 32),
    (5, 'Boris', 55),
    (6, 'Vera', 64), ],
    ['id', 'name', 'age'])
e = con.spark.createDataFrame([
    (1, 2, 'friend'),
    (2, 1, 'friend'),
    (1, 3, 'friend'),
    (3, 1, 'friend'),
    (2, 3, 'friend'),
    (3, 2, 'friend'),
    (4, 2, 'follows'),
    (5, 6, 'follows'), ],
    ['src', 'dst', 'type'])
g = GraphFrame(v, e)

In [8]:
g.vertices.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Anton| 23|
|  2| Anna| 27|
|  3|Andry| 24|
|  4| Alex| 32|
|  5|Boris| 55|
|  6| Vera| 64|
+---+-----+---+



In [9]:
g.vertices.rdd.getNumPartitions()

130

In [10]:
g.vertices.repartition(1, ['id'])

DataFrame[id: bigint, name: string, age: bigint]

In [11]:
con.repartitioning?

Signature:
con.repartitioning(
    df: pyspark.sql.dataframe.DataFrame,
    mode: str = 'parquet',
    partititon_cols: list = None,
    block_size: int = 134217728,
    serialization_factor: int = 8,
) -> pyspark.sql.dataframe.DataFrame
Docstring:
Метод для партиционирования датафрейма по заданному размеру
mode - может быть ['memory', 'parquet']. Запись на диск или кэширование в памяти.
block_size - размер блока в HDFS
serialization_factor - коэффициент, для оптимальной записи в parquet
File:      ~/work/Spark_conn/connector.py
Type:      method

In [12]:
con.repartition_and_checkpoint?

Signature:
con.repartition_and_checkpoint(
    df: pyspark.sql.dataframe.DataFrame,
    partititon_cols: list = None,
    cache: bool = True,
) -> pyspark.sql.dataframe.DataFrame
Docstring:
Метод для создания чекпоинта, оптимального по размеру партиций для дальнейшей работы.
partititon_cols - название колонки партиционирования или лист колонок.
cache - запись через оперативную память или через запись на диск.
File:      ~/work/Spark_conn/connector.py
Type:      method

In [13]:
con.read_table?

Signature: con.read_table(table: str, DB: str = None) -> pyspark.sql.dataframe.DataFrame
Docstring:
DB - путь к корневой папке в HDFS. Если задан при инициализации объекта, то можно указывать только путь к таблице.
table - путь к таблице относительно корневой папки в HDFS.
File:      ~/work/Spark_conn/connector.py
Type:      method

In [14]:
con.clickhouse_read_table?

Signature:
con.clickhouse_read_table(
    url: str,
    table: str,
    user: str,
    password: str,
    driver: str = 'com.clickhouse.jdbc.ClickHouseDriver',
) -> pyspark.sql.dataframe.DataFrame
Docstring:
url = "jdbc:clickhouse://<host>:<port>/<database>"
table = "<table_name>"
user = "<username>"
password = "<password>"
driver = "com.clickhouse.jdbc.ClickHouseDriver"
File:      ~/work/Spark_conn/connector.py
Type:      method

In [15]:
con.postgres_read_table?

Signature:
con.postgres_read_table(
    url: str,
    table: str,
    user: str,
    password: str,
    driver: str = 'org.postgresql.Driver',
) -> pyspark.sql.dataframe.DataFrame
Docstring:
url = путь/ДБ "jdbc:postgresql://<host>:<port>/<database>"
table = название таблицы
user = "postgres"
password = "password"
driver = "org.postgresql.Driver"
File:      ~/work/Spark_conn/connector.py
Type:      method

In [16]:
con.save_table?

Signature:
con.save_table(
    df: pyspark.sql.dataframe.DataFrame,
    table: str,
    save_mode: str = 'overwrite',
    partitionBy: list = None,
)
Docstring:
table - полный путь к таблице
save_mode - перезапись таблицы или добавление строк в существующую ['overwrite', 'append']
partitionBy - ключ партиционирования
File:      ~/work/Spark_conn/connector.py
Type:      method

In [17]:
con.clickhouse_save_table?

Signature:
con.clickhouse_save_table(
    df: pyspark.sql.dataframe.DataFrame,
    table: str,
    url: str,
    orderBy: str,
    user: str,
    password: str,
    save_mode: str = 'overwrite',
    driver: str = 'com.clickhouse.jdbc.ClickHouseDriver',
)
Docstring:
url = "jdbc:clickhouse://<host>:<port>/<database>"
table = "<table_name>"
orderBy = clickhouse MergeTree must be set order by (ID), where ID is orderBy arg
user = "<username>"
password = "<password>"
driver = "com.clickhouse.jdbc.ClickHouseDriver"
File:      ~/work/Spark_conn/connector.py
Type:      method

In [18]:
con.postgres_save_table?

Signature:
con.postgres_save_table(
    df: pyspark.sql.dataframe.DataFrame,
    table: str,
    url: str,
    user: str,
    password: str,
    save_mode: str = 'overwrite',
    driver: str = 'org.postgresql.Driver',
)
Docstring:
url = путь/ДБ "jdbc:postgresql://<host>:<port>/<database>"
table = название таблицы
user = "postgres"
password = "password"
driver = "org.postgresql.Driver"
File:      ~/work/Spark_conn/connector.py
Type:      method

In [19]:
con.del_Checkpoints?

Signature: con.del_Checkpoints() -> None
Docstring: Метод очищает папку с чекпоинтами
File:      ~/work/Spark_conn/connector.py
Type:      method

In [20]:
con.stop_spark?

Signature: con.stop_spark() -> None
Docstring: Метод закрывает сессию спарка и очищает папку с чекпоинтами
File:      ~/work/Spark_conn/connector.py
Type:      method

In [21]:
from clickhouse_driver import Client

client = Client(host='10.0.2.15', port=9000, user='utopialvo', password='utopialvo')

In [22]:
client

In [23]:
result = client.execute('SELECT 1')
print(result)

[(1,)]


In [44]:
client.execute('CREATE DATABASE IF NOT EXISTS testDB')

[]

In [24]:
client.execute('SELECT * FROM postgres_materialized_db.hacker_news LIMIT 1')

[(Decimal('11908254'),
  Decimal('0'),
  'story',
  'mechmind',
  datetime.datetime(2016, 6, 15, 10, 6, 17),
  '',
  Decimal('0'),
  Decimal('0'),
  Decimal('0'),
  ['1190923',
   '191230',
   '190966',
   '190866',
   '190888',
   '190868',
   '190934',
   '190955',
   '190957',
   '190854',
   '190898',
   '196853',
   '190913',
   '191183',
   '191033',
   '191175'],
  'https://clickhouse.yandex/reference_en.html',
  Decimal('243'),
  'ClickHouse – high-performance open-source distributed column-oriented DBMS',
  [],
  Decimal('70'))]

In [25]:
import psycopg

In [26]:
conn = psycopg.connect(
dbname="clickhouse_pg_db",
user="utopialvo",
password="utopialvo",
host="10.0.2.15",
port="5432"
)

In [27]:
cur = conn.cursor()

In [28]:
cur.execute('SELECT * FROM hacker_news LIMIT 1')

<psycopg.Cursor [TUPLES_OK] [INTRANS] (host=10.0.2.15 user=utopialvo database=clickhouse_pg_db) at 0x72095dab3e20>

In [29]:
cur.close()
conn.close()

In [30]:
con.postgres_read_table(url="jdbc:postgresql://10.0.2.15:5432/clickhouse_pg_db",table='hacker_news',user='utopialvo', password='utopialvo').printSchema()

root
 |-- id: decimal(38,18) (nullable = true)
 |-- deleted: decimal(38,18) (nullable = true)
 |-- type: string (nullable = true)
 |-- by: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- text: string (nullable = true)
 |-- dead: decimal(38,18) (nullable = true)
 |-- parent: decimal(38,18) (nullable = true)
 |-- poll: decimal(38,18) (nullable = true)
 |-- kids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- url: string (nullable = true)
 |-- score: decimal(38,18) (nullable = true)
 |-- title: string (nullable = true)
 |-- parts: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- descendants: decimal(38,18) (nullable = true)



In [31]:
data = con.postgres_read_table(url="jdbc:postgresql://10.0.2.15:5432/clickhouse_pg_db",table='hacker_news',user='utopialvo', password='utopialvo')

In [32]:
data.select('title','time').distinct().show()

+--------------------+-------------------+
|               title|               time|
+--------------------+-------------------+
|                    |2020-10-20 15:58:07|
|                    |2019-11-23 13:58:53|
|                    |2021-05-11 08:58:18|
|                    |2021-09-20 22:25:29|
|ClickHouse has ra...|2021-03-15 15:47:29|
|                    |2021-09-20 17:42:47|
|                    |2019-11-03 10:03:34|
|                    |2020-05-06 08:12:56|
|                    |2021-05-17 07:23:39|
|                    |2021-05-17 10:29:40|
|                    |2020-06-01 20:41:53|
|                    |2020-04-06 20:49:44|
|                    |2021-03-22 10:25:57|
|                    |2018-04-11 14:52:34|
|                    |2019-08-13 11:48:08|
|                    |2020-03-02 19:49:32|
|                    |2019-06-12 17:59:06|
|                    |2021-07-28 23:21:04|
|                    |2020-08-26 23:34:33|
|                    |2019-05-15 03:07:34|
+----------

In [33]:
con.postgres_save_table(df=data.select('title','time').distinct(), table='test_table', url="jdbc:postgresql://10.0.2.15:5432/clickhouse_pg_db",user='utopialvo', password='utopialvo')

In [34]:
con.postgres_read_table(url="jdbc:postgresql://10.0.2.15:5432/clickhouse_pg_db",table='test_table',user='utopialvo', password='utopialvo').printSchema()

root
 |-- title: string (nullable = true)
 |-- time: timestamp (nullable = true)



In [35]:
data = con.postgres_read_table(url="jdbc:postgresql://10.0.2.15:5432/clickhouse_pg_db",table='test_table',user='utopialvo', password='utopialvo')

In [36]:
data

DataFrame[title: string, time: timestamp]

In [39]:
data = data.checkpoint()

In [40]:
data

DataFrame[title: string, time: timestamp]

In [45]:
con.clickhouse_save_table(df= data, table='test_table', url="jdbc:clickhouse://10.0.2.15:8123/testDB", orderBy='time', user='utopialvo', password='utopialvo')

In [47]:
data = con.clickhouse_read_table(url="jdbc:clickhouse://10.0.2.15:8123/testDB", table='test_table', user='utopialvo', password='utopialvo')

In [48]:
data.printSchema()

root
 |-- title: string (nullable = true)
 |-- time: timestamp (nullable = true)



In [49]:
#con.stop_spark()

time: 2024-12-21 10:59:30.610229
del
stop
